In [ ]:
# 조이스틱 젯봇 -> 본체로 옮기기

### ✅ 무선으로 조이스틱을 활용해보자!
- 젯봇 조종, 카메라도 같이 활용
- http://html5gamepad.com 사이트 접속

In [2]:
import ipywidgets.widgets as widgets

from IPython.display import display

controller = widgets.Controller(index = 0)

display(controller)

Controller()

In [13]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg

import traitlets

camera = Camera.instance(width = 224, height = 224)
image = widgets.Image(width = 224, height = 224, format = 'jpeg')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform = bgr8_to_jpeg)
# 디지털 링크를 생성해서 카메라 입력과 이미지 위젯 연결!

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [5]:
# 카메라 끄기

camera_link.unlink()

In [ ]:
# 조이스틱 조종, 카메라 활용!
# 두가지 일을 동시에 처리해야 함! -> Thread 라이브러리 활용해서 동시 작업!

In [3]:
# 숫자를 출력하는 함수!

import time

def count_number(start, end) :
    for i in range(start, end + 1) :
        print(i, end = " ")
        time.sleep(0.5)
    print()

In [4]:
count_number(1, 10)
count_number(101, 110) # 하나가 끝나야 실행! -> 함수는 두개를 동시에 할 수 없음

1 2 3 4 5 6 7 8 9 10 
101 102 103 104 105 106 107 108 109 110 


### ✅ Thread 라이브러리 활용하기!
- threading을 임포트
- threading.Thread(target = 함수, args = (범위 지정))

In [9]:
import threading

thread1 = threading.Thread(target = count_number, args = (1, 100))
thread2 = threading.Thread(target = count_number, args = (10, 100))

thread1.start()
thread2.start()

1 10 2 11 3 12 13 4 14 5 

In [7]:
# 외부에서 Thread 능동적으로 멈추는 함수 정의

import inspect # inspect -> 코드 내의 객체에 대한 정보를 검사하는 데 사용
import ctypes # C언어 타입 / ctypes 모듈은 C 라이브러리를 사용하기 위한 외부 함수 호출을 제공하며, 이 때 C 타입을 다루기 위해 사용 

# stop_thread(멈출 쓰레드)
# 이 함수를 실행하면 쓰레드가 멈춘다!!

def _async_raise(tid, exctype): #쓰레드에게 예외를 발생시키는 역할
    
    """raises the exception, performs cleanup if needed"""
    
    tid = ctypes.c_long(tid) #쓰레드 식별자를 C 타입인 c_long으로 변환
    
    if not inspect.isclass(exctype): #exctype가 클래스가 아니면
        exctype = type(exctype) # 해당 객체의 타입을 가져와서 클래스로 사용
        
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype)) #함수를 사용하여 쓰레드에게 예외를 발생 -> 반환값은 성공 여부
    
    if res == 0:#반환값이 0이면
        raise ValueError("invalid thread id") #쓰레드 식별자가 유효하지 않다는 예외를 발생
    elif res != 1:#반환값이 1이 아니면 
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None) #쓰레드에게 예외를 다시 발생시켜서 효과를 되돌림
        
def stop_thread(thread): # 주어진 쓰레드를 강제로 종료 
    _async_raise(thread.ident, SystemExit) #SystemExit 예외를 발생시킴으로써 쓰레드를 종료

35 44 36 45 37 46 38 47 

In [10]:
stop_thread(thread1)
stop_thread(thread2)

In [12]:
import custom
import head

def jetbot_motion():
    #[1] -> 조이스틱의 왼쪽 위아래!
    
    while 1:
        
        if controller.axes[1].value == 1:
            custom.backward()
        elif controller.axes[1].value == -1:
            custom.forward()
            
        #[0]버튼 -> 조이스틱의 1번버튼
        
        if controller.buttons[0].value == True:
            custom.stop()
        if controller.buttons[1].value == True:
            head.center()
        if controller.buttons[2].value == True:
            head.pan_right()

serial Open!


In [14]:
import threading

thread3 = threading.Thread(target = jetbot_motion)

thread3.start()